## Report trend analysis for past *n* days

Import necessary packages

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

Analyze trends

In [ ]:
# Read files from folder
folder = '../daily_prices'
files = os.listdir(folder)

# Create dataframe
seasonal_uplifts = dict()

for i, file in enumerate(files):
    # Read file
    df = pd.read_csv(os.path.join(folder, file))

    if not df.isna().any().any() and len(df.index) >= 504:
        # Decompose
        result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
        trend = result.trend[-20:].reset_index(drop=True)

        # normalize input data
        mins, maxs = trend.min(), trend.max()
        trend = (trend-mins)/(maxs-mins)

        # Get seasonality for next month (20 trading days)
        if past:
            seasonality = seasonal[-252:][-20:].reset_index(drop=True)
        else:
            seasonality = seasonal[-252:][:20].reset_index(drop=True)

        # Add to dataframe
        seasonal_uplifts[file[:-4]] = seasonality.to_list()

seasonal_uplifts = pd.DataFrame(seasonal_uplifts)